In [1]:
import run_simulation

In [2]:
run_simulation.main()

PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\NEC_59~1\AppData\Local\Temp\pyaedt_NEC_5950X1_b0f0768a-db43-4caa-8aed-abb218e46f01.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 63633.
PyAEDT ERROR: Error getting status: Cannot connect to license server system. (-15,10:10061 "WinSock: Connection refused")
PyAEDT WARNING: Electronics Desktop license not found on the default license server.
PyAEDT INFO: Electronics Desktop started on gRPC port: 63633 after 9.078136205673218 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.Elec

In [1]:
import run_simulation

# 1. Simulation 클래스의 인스턴스를 생성합니다.
simulation_runner = run_simulation.Simulation()

# 2. create_design 메서드를 호출하여 프로젝트와 디자인을 초기화합니다.
# 이 과정에서 내부에 self.design 객체가 생성됩니다.
simulation_runner.create_design("SST_MFT")

# 3. 이제 입력 매개변수를 생성할 수 있습니다.
# 이 메서드는 초기화된 design 객체를 사용하게 됩니다.
input_parameters = simulation_runner.create_input_parameter()

# 4. 생성된 파라미터를 Simulation 객체와 Ansys 디자인에 설정합니다.
simulation_runner.set_variable(input_parameters)

# 5. 해석 설정 및 실행
simulation_runner.set_maxwell_analysis()

# 6. 모델을 생성합니다.
simulation_runner.create_core()
simulation_runner.create_face(simulation_runner.maxwell_design)
simulation_runner.create_windings()
simulation_runner.create_mold()
simulation_runner.create_cold_plate()
simulation_runner.create_air()

# 7. 메쉬 및 경계조건 설정
simulation_runner.assign_meshing()
simulation_runner.assign_excitations()

# 8. 해석 설정 및 실행
simulation_runner.analyze_maxwell()

# 9. 결과 리포팅
simulation_runner.get_simulation_results(input=True)

# 10. 두 번째 해석 실행
simulation_runner.second_simulation()

# 11. Icepak 디자인 생성
simulation_runner.create_icepak()       

# 12. Icepak 해석 설정
simulation_runner.setup_icepak_analysis()

# 13. Icepak 해석 실행
simulation_runner.analyze_icepak()

# 14. Icepak 결과 리포팅
simulation_runner.get_icepak_results()

# 15. 결과 저장
simulation_runner.save_results_to_csv(simulation_runner.results_df)




PyAEDT INFO: Python version 3.11.11 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:34:19) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\NEC_59~1\AppData\Local\Temp\pyaedt_NEC_5950X1_3c9d7b49-aa62-4d58-adae-56e2dea08d31.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 63320.
PyAEDT ERROR: Error getting status: Cannot connect to license server system. (-15,10:10061 "WinSock: Connection refused")
PyAEDT WARNING: Electronics Desktop license not found on the default license server.
PyAEDT INFO: Electronics Desktop started on gRPC port: 63320 after 9.07043194770813 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.Elect

In [4]:
parameters = [
    [simulation_runner.maxwell_design.winding1, f"P_winding1", "EMLoss"],
    [simulation_runner.maxwell_design.winding2, f"P_winding2", "EMLoss"],
    [simulation_runner.maxwell_design.core, f"P_Core", "CoreLoss"],
    [simulation_runner.maxwell_design.leg_left , f"B_mean_leg_left", "B_mean"],
    [simulation_runner.maxwell_design.leg_right, f"B_mean_leg_right", "B_mean"],
    [simulation_runner.maxwell_design.leg_top, f"B_mean_leg_top", "B_mean"],
    [simulation_runner.maxwell_design.leg_bottom, f"B_mean_leg_bottom", "B_mean"],
]

name_list = []
result_expressions = []
report = None

result_expressions, name_list = simulation_runner.icepak_design._add_calculator_expression(parameters=parameters)

AttributeError: 'Icepak' object has no attribute '_get_calculator_loss'

In [ ]:
def get_calculator_parameter(self, dir=None, parameters=[], mod="write", import_report=None) :
        """
        example :
        parameters2 = []
        parameters2.append([winding1, "P_LV", "EMLoss"])
        # ... (rest of example)
        """
        name_list = []
        result_expressions = []
        report = None

        if mod == "write" :
            result_expressions, name_list = self._add_calculator_expression(parameters=parameters)
            report = self._create_report(report_name = "calculator_report", result_expressions = result_expressions, category = "Fields")
            if not hasattr(self, "report_list"):
                self.report_list = {}
            self.report_list["icepak_calc_report"] = report
        elif mod == "read" :
            report = self.report_list.get("icepak_calc_report", import_report)
            # In "read" mode, reconstruct the expression and name lists for column mapping
            for obj, name, expression_type in parameters:
                name_list.append(name)
                obj_name = obj.name if hasattr(obj, 'name') else obj
                
                if expression_type == "Temp_max":
                    expr_name = f"Temp_max_{obj_name}"
                elif expression_type == "Temp_mean":
                    expr_name = f"Temp_mean_{obj_name}"
                elif expression_type == "B_mean":
                    expr_name = f"B_mean_{obj_name}"
                else: 
                    expr_name = f"P_{obj_name}"
                result_expressions.append(expr_name)
        
        if not report:
             return None, pd.DataFrame(columns=[p[1] for p in parameters])

        export_data = self.post.export_report_to_csv(project_dir=dir, plot_name=report.plot_name)
        data = pd.read_csv(export_data)

        # Create a mapping from the actual column names in the CSV to the desired names
        rename_mapping = {}
        for expr, desired_name in zip(result_expressions, name_list):
            for col in data.columns:
                if expr in col:
                    rename_mapping[col] = desired_name
                    break
        
        # Rename columns and select only the ones we need
        output_df = data.rename(columns=rename_mapping)

        # Ensure all desired columns exist, adding missing ones with NaN
        for name in name_list:
            if name not in output_df.columns:
                output_df[name] = np.nan

        output_df = output_df[name_list]
        output_df.dropna(inplace=True)

        output_df.to_csv("icepak_calculator.csv")
        
        return report, output_df

In [ ]:
import os

from report import (
    get_input_parameter, get_maxwell_magnetic_parameter,
    get_maxwell_calculator_parameter, get_convergence_report, get_icepak_calculator_parameter
)



current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "simulation", f"{simulation_runner.PROJECT_NAME}")
        
        # Get results using the calculator
# icepak_results_df = get_icepak_calculator_parameter(simulation_runner.icepak_design, dir=folder_path)  

simulation_runner.icepak_design.winding1

In [ ]:

params = [
    [design.core, f"Temp_max_core", "Temp_max"],
    [design.core, f"Temp_mean_core", "Temp_mean"],
    [design.winding1, f"Temp_max_winding1", "Temp_max"],
    [design.winding1, f"Temp_mean_winding1", "Temp_mean"],
    [design.winding2, f"Temp_max_winding2", "Temp_max"],
    [design.winding2, f"Temp_mean_winding2", "Temp_mean"],
    [design.leg_left, f"Temp_max_leg_left", "Temp_max"],
    [design.leg_right, f"Temp_max_leg_right", "Temp_max"],
    [design.leg_top, f"Temp_max_leg_top", "Temp_max"],
    [design.leg_bottom, f"Temp_max_leg_bottom", "Temp_max"],
    [design.leg_left, f"Temp_mean_leg_left", "Temp_mean"],
    [design.leg_right, f"Temp_mean_leg_right", "Temp_mean"],
    [design.leg_top, f"Temp_mean_leg_top", "Temp_mean"],
    [design.leg_bottom, f"Temp_mean_leg_bottom", "Temp_mean"],
]

df = design.get_calculator_parameter(dir=dir, parameters=params)





In [5]:
simulation_runner.w1

np.float64(223.4)

In [ ]:
simulation_runner.winding1.faces

In [ ]:
input_parameter = sim1.create_input_parameter()

sim1.set_variable(input_parameter)


In [ ]:
sim1.create_core()


In [6]:
def offset_calculation(coil_diameter, h1, height_ratio, offset_z) :

    plus = coil_diameter/2
    minus = ((h1*height_ratio) - coil_diameter/2)

    offset = (plus + minus)/2 - plus + offset_z
    offset = (plus + minus)/2 - plus

    return offset

offset1 = offset_calculation(sim1.N1_coil_diameter, sim1.h1, sim1.N1_height_ratio, sim1.N1_offset)
winding_params1 = {
    "N" : sim1.N1,
    "N_layer" : sim1.N1_layer,
    "x" : "w1 + 2*N1_space_w",
    "y" : "l1 + 2*N1_space_l",
    "coil_diameter" : "N1_coil_diameter",
    "coil_zgap" : "N1_coil_zgap",
    "coil_layer_x_gap" : "N1_layer_gap",
    "coil_layer_y_gap" : "N1_layer_gap",
    "color" : [255, 50, 50],
    "transparency" : 0,
    "offset" : ["0mm", "-(l1+l2)/2", f"{offset1}mm"],
    "terminal_position" : "w1/2 + 150mm",
    "Num" : 18
}
sim1.create_winding(name="winding1", **winding_params1)

offset2 = offset_calculation(sim1.N2_coil_diameter, sim1.h1, sim1.N2_height_ratio, sim1.N2_offset)
winding_params2 = {
    "N" : sim1.N2,
    "N_layer" : sim1.N2_layer,
    "x" : "w1 + 2*N2_space_w",
    "y" : "l1 + 2*N2_space_l",
    "coil_diameter" : "N2_coil_diameter",
    "coil_zgap" : "N2_coil_zgap",
    "coil_layer_x_gap" : "N2_layer_gap",
    "coil_layer_y_gap" : "N2_layer_gap",
    "color" : [50, 50, 255],
    "transparency" : 0,
    "offset" : ["0mm", "(l1+l2)/2", f"{offset2}mm"],
    "terminal_position" : "w1/2 + 150mm",
    "Num" : 18
}
sim1.create_winding(name="winding2", **winding_params2)